<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/LR_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector
 

In [2]:
# Loading data
# file_path = Path("RHT.V.csv")
# if(file_path=='RHT.V.csv'):

df = pd.read_csv("ORCL.csv")
df.dropna(axis=0,inplace=True)

df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,78.760002,78.870003,78.059998,78.099998,7295900,0.049384,-0.837993,Negative,Positive,Negative,0,Positive
750,24/06/2021,78.650002,78.959999,77.610001,77.739998,9624000,0.096294,-1.157029,Negative,Negative,Negative,0,Negative
751,25/06/2021,78.029999,78.610001,77.559998,78.459999,12845000,0.857142,0.551071,Positive,Negative,Positive,0,Negative
752,28/06/2021,78.489998,78.970001,78.330002,78.500000,5605400,0.265622,0.012743,Negative,Positive,Positive,0,Positive
753,29/06/2021,78.540001,78.589996,77.820000,78.099998,5392900,0.363636,-0.560227,Negative,Negative,Negative,0,Positive


In [3]:
df['FutureTrend'].value_counts()

Negative    412
Positive    340
Neutral       2
Name: FutureTrend, dtype: int64

In [4]:
# Setting Index as Date
appl_df = df.set_index("Date")
appl_df.tail()
#drop na
appl_df.dropna(axis=0,inplace=True)

In [5]:
def make_bins(df):
   label_names = ["Ef_1_v_low","Ef_1_low","Ef_1_v_med","Ef1_high", "Ef1_very_high" ]
   cut_points =[0. , 0.2, 0.4, 0.6, 0.8, 1. ]
   df["EF_1_val"] = pd.cut(df["EF_1"], cut_points, labels=label_names)
   dummies = pd.get_dummies(df["EF_1_val"],drop_first=True)
   df = pd.concat([df, dummies], axis=1)
   label_names = ["setinent_neutral","sentiment_good","sentiment_bad"]
   cut_points =[0, 0.29,0.58 ,0.87] 
   df["sentiments"]=pd.cut(df['Twitter Sentiments'], cut_points, labels=label_names)
   dummies1 = pd.get_dummies(df["sentiments"],drop_first=True)
   df = pd.concat([df, dummies1], axis=1)
   df.drop(["EF_1_val","sentiments"],axis=1,inplace=True)
   return df

def calculate_ema(prices, days, smoothing=2):
      ema = [sum(prices[:days]) / days]
      for price in prices[days:]:
          ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
      return ema

In [6]:
appl_df.head()

,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
Date,,,,,,,,,,,,
02/07/2018,43.959999,44.970001,43.770000,44.950001,20655500,0.983333,2.252051,Positive,Negative,Positive,85,Positive
03/07/2018,44.990002,44.990002,44.520000,44.720001,16783300,0.425532,-0.600134,Neutral,Neutral,Negative,60,Negative
05/07/2018,45.150002,45.470001,44.990002,45.410000,21916200,0.874997,0.575855,Negative,Positive,Positive,70,Negative
06/07/2018,45.459999,46.189999,45.270000,46.000000,20764300,0.793479,1.187859,Negative,Negative,Positive,43,Negative
09/07/2018,46.130001,46.790001,46.060001,46.730000,25125700,0.917806,1.300669,Negative,Positive,Positive,62,Negative


In [7]:
target=appl_df['FutureTrend']
print(appl_df.head())
appl_df=appl_df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
appl_df = make_bins(df)
appl_df.head(2)
appl_df=appl_df.drop(["Twitter Sentiments"],axis =1)
appl_df['diff']=appl_df['Close']-appl_df['Open']
appl_df['10day']=appl_df['Close'].rolling(10).mean()
appl_df['10day'].fillna(appl_df['10day'][9],inplace=True)
  
ema = calculate_ema(appl_df['Close'], 10)
appl_df=appl_df[9:]
target=target[9:]
appl_df['ema']=ema
appl_df['signal']=appl_df['ema']>appl_df['Close']

del (appl_df['Open'])
del (appl_df['Close'])
del (appl_df['High'])
del (appl_df['EF_2'])

appl_df = appl_df.set_index("Date")

# appl_df['normVol'] = appl_df['Volume'] / appl_df['Volume'].ewm(5).mean()   

appl_df.head()

                 Open       High        Low      Close    Volume      EF_1  \
Date                                                                         
02/07/2018  43.959999  44.970001  43.770000  44.950001  20655500  0.983333   
03/07/2018  44.990002  44.990002  44.520000  44.720001  16783300  0.425532   
05/07/2018  45.150002  45.470001  44.990002  45.410000  21916200  0.874997   
06/07/2018  45.459999  46.189999  45.270000  46.000000  20764300  0.793479   
09/07/2018  46.130001  46.790001  46.060001  46.730000  25125700  0.917806   

                EF_2 Oil FutureTrend Gold FutureTrend     Trend  \
Date                                                              
02/07/2018  2.252051        Positive         Negative  Positive   
03/07/2018 -0.600134         Neutral          Neutral  Negative   
05/07/2018  0.575855        Negative         Positive  Positive   
06/07/2018  1.187859        Negative         Negative  Positive   
09/07/2018  1.300669        Negative         Positi

,Low,Volume,EF_1,Oil FutureTrend,Gold FutureTrend,Trend,FutureTrend,Ef_1_low,Ef_1_v_med,Ef1_high,Ef1_very_high,sentiment_good,sentiment_bad,diff,10day,ema,signal
Date,,,,,,,,,,,,,,,,,
16/07/2018,47.950001,22949600,0.761194,Negative,Positive,Negative,Negative,0,0,1,0,0,0,-0.160000,46.768,46.768000,False
17/07/2018,48.150002,18823100,0.892857,Negative,Negative,Positive,Positive,0,0,0,1,0,0,0.610001,47.163,47.155637,False
18/07/2018,48.400002,19234700,0.399997,Negative,Positive,Negative,Positive,1,0,0,0,0,0,-0.150002,47.555,47.425521,False
19/07/2018,48.209999,16656500,0.450978,Negative,Negative,Positive,Negative,0,1,0,0,0,0,0.039997,47.858,47.609972,False
20/07/2018,48.360001,17232900,0.457144,Positive,Positive,Positive,Negative,0,1,0,0,0,0,0.029999,48.110,47.775431,False


In [8]:
new_df=appl_df

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
   #('ordinal-encoder', categorical_preprocessor, categorical_columns),
    ('minmax-scaler', numerical_preprocessor, numerical_columns)])



['Oil FutureTrend', 'Gold FutureTrend', 'Trend', 'FutureTrend']


train_test splt

In [9]:

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
model_lr = make_pipeline(preprocessor,LogisticRegression(solver='newton-cg', class_weight='balanced'))
model = make_pipeline(preprocessor,  RandomForestClassifier(n_estimators=30,
                                      max_features=2,
                                      max_depth=3,
                                      random_state=0))

In [ ]:
X_train

,Low,Volume,EF_1,Oil FutureTrend,Gold FutureTrend,Trend,FutureTrend,Ef_1_low,Ef_1_v_med,Ef1_high,Ef1_very_high,sentiment_good,sentiment_bad,diff,10day,ema,signal
Date,,,,,,,,,,,,,,,,,
25/10/2018,125.139999,10304300,0.494338,Negative,Negative,Negative,Positive,0,1,0,0,0,0,-0.800003,133.569003,132.950287,True
16/05/2019,134.639999,3017300,0.843540,Positive,Positive,Positive,Positive,0,0,0,1,0,0,1.139999,136.194000,135.885201,True
25/06/2020,116.099998,6130600,0.942122,Positive,Negative,Positive,Positive,0,0,0,1,0,0,2.269997,121.542001,121.321037,True
12/03/2020,102.279999,12512700,0.070479,Negative,Positive,Negative,Negative,0,0,0,0,0,0,-6.840004,124.820999,123.492499,True
12/10/2018,139.080002,6563400,0.719516,Negative,Negative,Negative,Negative,0,0,1,0,0,0,-0.269989,147.850002,145.800965,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24/10/2018,127.000000,5710300,0.044776,Negative,Negative,Negative,Positive,0,0,0,0,0,0,-3.959999,134.826003,134.394796,True
14/12/2018,119.440002,4554200,0.275449,Positive,Negative,Positive,Positive,1,0,0,0,0,0,0.400002,121.824000,121.048496,True
12/08/2019,133.179993,4083900,0.182158,Negative,Positive,Negative,Negative,0,0,0,0,0,0,-1.990005,142.608002,140.930683,True


In [ ]:
target.tail()


In [10]:
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.1, random_state=42)

In [11]:
model_lr.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmax-scaler',
                                                  StandardScaler(),
                                                  ['Low', 'Volume', 'EF_1',
                                                   'Ef_1_low', 'Ef_1_v_med',
                                                   'Ef1_high', 'Ef1_very_high',
                                                   'sentiment_good',
                                                   'sentiment_bad', 'diff',
                                                   '10day', 'ema',
                                                   'signal'])])),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced',
                                    solver='newton-cg'))])

In [12]:
  # Scoring the model
print("Accuracy score (training): {0:.3f}".format(
        model_lr.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_lr.score(
            X_test, 
            y_test)))


Accuracy score (training): 0.536
Accuracy score (validation): 0.493
